# ML experimentation with MLflow via private VPC


This notebook is used to make sure that your setup is ready to run for the workshop. 
The main steps are
* Set constants that are used throughout the workshop, e.g. `project_prefix`, `bucket_prefix`, etc
* Set Mlflow server `mlflow_arn` that will be used throughout the workshop
* Install Docker package to enable Studio local mode

It also provides you instructions how to resolve issues, in case you are running this on your own AWS account.

To run this notebook and all notebooks in the workshop please use the `Python 3` kernel in JupyterLab

Before starting, let us first setup the private `pip` repository via code artifact and get the authentication token to install the libraries.

In [ ]:
%%bash
AWS_ACCOUNT=$(aws sts get-caller-identity --output text --query 'Account')
aws codeartifact login --tool pip --repository pip --domain code-artifact-domain --domain-owner ${AWS_ACCOUNT} --region ${AWS_DEFAULT_REGION}

## Initiate
Get the latest version of SageMaker Python SDK.

<div class="alert alert-info"> 💡 This notebook was tested with Sagemaker Distribution `2.1.0` and the SageMaker Python SDK version 2.227.0. The notebooks don't pin the version of the sagemaker. If you encounter any incompatibility issues, you can install the specific version of the sagemaker by running the pip command: <code>!pip install sagemaker=2.227.0</code>
</div>

Initialize the reference to the private AWS CodeArtifact to install the relevant libraries

### Let's get started by installing the requirements.

In [ ]:
%%writefile ./requirements.txt
sagemaker
scikit-learn==1.3.2
s3fs
mlflow==2.13.2
sagemaker-mlflow

In [ ]:
!pip install -r requirements.txt

### Import packages

In [ ]:
import time
import os
import json
import boto3
import numpy as np  
import pandas as pd 
import sagemaker
from time import gmtime, strftime, sleep

(sagemaker.__version__,boto3.__version__)

### Set constants

In [ ]:
# Get some variables you need to interact with SageMaker service
boto_session = boto3.Session()
region = boto_session.region_name
project_prefix = "amzn"
bucket_name = sagemaker.Session().default_bucket()
bucket_prefix = f"{bucket_name}/{project_prefix}"
sm_session = sagemaker.Session()
sm_client = boto_session.client("sagemaker")
sm_role = sagemaker.get_execution_role()
account_id = sm_role.split(':')[-2]

print(sm_role)

### Get domain id
You need this value `domain_id` in many SageMaker Python SDK and boto3 SageMaker API calls. The notebook metadata file contains `domain_id` value. The following code demonstrates how to access the notebook metadata file and get the `domain_id`.

In [ ]:
NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        metadata = json.loads(f.read())
        domain_id = metadata.get('DomainId')
        space_name = metadata.get('SpaceName')
        print(f"SageMaker domain id: {domain_id}")

if not space_name:
    raise Exception(f"Cannot find the current space name. Make sure you run this notebook in a JupyterLab in the SageMaker Studio")
else:
    print(f"Space name: {space_name}")
    
r = sm_client.describe_space(DomainId=domain_id, SpaceName=space_name)
user_profile_name = r['OwnershipSettings']['OwnerUserProfileName']

assert(user_profile_name)
print(f"User profile: {user_profile_name}")

### Connect to MLflow tracking server

Fetch the `ARN` of the MLflow server (if you have multiple MLflow Tracking Servers, you can just the `mlflow_arn` and `mlflow_name` to the value of your server).

In [ ]:
# Find an active MLflow server in the account
r = sm_client.list_mlflow_tracking_servers(
    TrackingServerStatus='Created',
)['TrackingServerSummaries']

mlflow_arn = r[0]['TrackingServerArn']
mlflow_name = r[0]['TrackingServerName']

In [ ]:
(mlflow_arn, mlflow_name)

## Experimentation

In [ ]:
import xgboost

The Amazon SageMaker Python SDK supports setting of default values for AWS infrastructure primitive types, such as instance types, Amazon S3 folder locations, and IAM roles. You can override the default locations of these files by setting the SAGEMAKER_USER_CONFIG_OVERRIDE environment variables for the user-defined configuration file paths.

In [ ]:
domain_details = sm_client.describe_domain(DomainId=domain_id)
security_group_id = domain_details['DefaultUserSettings']['SecurityGroups'][0]
private_subnet_id_1 = domain_details['SubnetIds'][0]
private_subnet_id_2 = domain_details['SubnetIds'][1]

In [ ]:
config_yaml = f"""
SchemaVersion: '1.0'
SageMaker:
  PythonSDK:
    Modules:
      TelemetryOptOut: true
      RemoteFunction:
        # role arn is not required if in SageMaker Notebook instance or SageMaker Studio
        # Uncomment the following line and replace with the right execution role if in a local IDE
        # RoleArn: <replace the role arn here>
        S3RootUri: s3://{bucket_prefix}
        InstanceType: ml.m5.xlarge
        Dependencies: ./requirements.txt
        IncludeLocalWorkDir: true
        PreExecutionCommands:
        - "aws codeartifact login --tool pip --repository pip --domain code-artifact-domain --domain-owner {account_id} --region {region}"
        CustomFileFilter:
          IgnoreNamePatterns:
          - "data/*"
          - "models/*"
          - "*.ipynb"
          - "__pycache__"
        VpcConfig:
          SecurityGroupIds: 
          - {security_group_id}
          Subnets: 
          - {private_subnet_id_1}
          - {private_subnet_id_2}

"""

print(config_yaml, file=open('config.yaml', 'w'))
print(config_yaml)

In [ ]:
from time import gmtime, strftime

# Mlflow (replace these values with your own, if needed)
project_prefix = project_prefix
tracking_server_arn = mlflow_arn
experiment_name = f"{project_prefix}-sm-private-experiment"
run_name=f"run-{strftime('%d-%H-%M-%S', gmtime())}"

In [ ]:
import os

# Use the current working directory as the location for SageMaker Python SDK config file
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

### Data preparation

Due to the lack of internet access on the SageMaker Sudio domain, you need to download the reference dataset from public [UC Irvine ML repository](https://archive.ics.uci.edu/ml/machine-learning-databases/00601/ai4i2020.csv) to your local pc, then name it `predictive_maintenance_raw_data_header.csv` and upload the reference dataset from your local PC to JupyterLab Space as shown here.

![upload-to-jupyterlab](../image/upload-to-jupyterlab.png)

In [ ]:
input_data_dir = './'
if not os.path.exists(input_data_dir):
    os.makedirs(input_data_dir)
input_data_path = os.path.join(input_data_dir, 'predictive_maintenance_raw_data_header.csv')

### Data Processing

Determine the number of samples (rows) and features (columns) in the dataset.

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv(input_data_path)

print('The shape of the dataset is:', df.shape)

In [ ]:
df.head(10)

You will run data pre-processing in the preprocess function in the following cell. This function performs one-hot encoding of the relevant categorical columns and fills in the NaN values based on domain knowledge. It then splits the dataset into training, validation, and test datasets, fits the featurizer model, and transforms the datasets. The function returns the model and the output datasets, and saves the serialized model to the file system.

The following cell annotates the preprocess function with the @remote decorator to run the Python function as a SageMaker job without requiring any other modifications to the function code. Feel free to comment out the remote decorator in the cells below to seamlesssly move from running the function remotely via SageMaker Training to local execution. If you comment out the decorator to run the function locally, you will need to run this command in the terminal to give permission to the output directory where the function will save the models: sudo chmod -R 777 /opt/ml/model. You don't need to run this command if you leave the remote decorator in, since the config.yaml file runs that command before executing the training job.

The code also uses SageMaker Managed Warm Pools by setting the keep_alive_period_in_seconds parameter. SageMaker Managed Warm Pools let you retain and reuse provisioned infrastructure after the completion of a job to reduce latency for repetitive workloads, such as iterative experimentation or running many jobs consecutively. Subsequent training jobs that match specified parameters run on the retained warm pool infrastructure, which speeds up start times by reducing the time spent provisioning resources. Please note that Managed Warm Pools might not be enabled for your AWS Account; in such case, although the code will still work, you might not see lower latencies for the subsequent iterations.

In [ ]:
import os
import joblib

from sagemaker.workflow.execution_variables import ExecutionVariables
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


from sagemaker.remote_function import remote

import mlflow
import pandas as pd

@remote(keep_alive_period_in_seconds=3600, job_name_prefix=f"{project_prefix}-sm-private-preprocess")
def preprocess(df, df_source: str, experiment_name: str):
    
    mlflow.set_tracking_uri(tracking_server_arn)
    mlflow.set_experiment(experiment_name)    
    
    with mlflow.start_run(run_name=f"Preprocessing") as run:            
        mlflow.autolog()
        
        columns = ['Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Machine failure']
        cat_columns = ['Type']
        num_columns = ['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]']
        target_column = 'Machine failure'                    
        df = df[columns]

        mlflow.log_input(
            mlflow.data.from_pandas(df, df_source, targets=target_column),
            context="DataPreprocessing",
        )
        
        training_ratio = 0.8
        validation_ratio = 0.1
        test_ratio = 0.1
    
        X = df.drop(target_column, axis=1)
        y = df[target_column]
    
        print(f'Splitting data training ({training_ratio}), validation ({validation_ratio}), and test ({test_ratio}) sets ')
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=0, stratify=y)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_ratio/(validation_ratio+training_ratio), random_state=2, stratify=y_train)
    
        # Apply transformations
        transformer = ColumnTransformer(transformers=[('numeric', StandardScaler(), num_columns),
                                                      ('categorical', OneHotEncoder(), cat_columns)],
                                        remainder='passthrough')
        featurizer_model = transformer.fit(X_train)
        X_train = featurizer_model.transform(X_train)
        X_val = featurizer_model.transform(X_val)
    
        print(f'Shape of train features after preprocessing: {X_train.shape}')
        print(f'Shape of validation features after preprocessing: {X_val.shape}')
        print(f'Shape of test features after preprocessing: {X_test.shape}')
        
        y_train = y_train.values.reshape(-1)
        y_val = y_val.values.reshape(-1)
        
        print(f'Shape of train labels after preprocessing: {y_train.shape}')
        print(f'Shape of validation labels after preprocessing: {y_val.shape}')
        print(f'Shape of test labels after preprocessing: {y_test.shape}')
    
        model_file_path="/opt/ml/model/sklearn_model.joblib"
        os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
        joblib.dump(featurizer_model, model_file_path)

    return X_train, y_train, X_val, y_val, X_test, y_test, featurizer_model

The function returns multiple values, including the training, validation, and test features and labels, and the featurizer model.

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test, featurizer_model = preprocess(df=df, 
                                                                              df_source=input_data_path, 
                                                                              experiment_name=experiment_name)

Analyze the featurizer model structure.

In [ ]:
featurizer_model

In [ ]:
import pandas as pd
pd.DataFrame(X_train).head(10)

### Model Training

In [ ]:
import os
import xgboost
import numpy as np
import mlflow
from sagemaker.remote_function import remote

@remote(keep_alive_period_in_seconds=3600, job_name_prefix=f"{project_prefix}-sm-private-train")
def train(X_train, y_train, X_val, y_val,
          eta=0.1, 
          max_depth=2, 
          gamma=0.0,
          min_child_weight=1,
          verbosity=0,
          objective='binary:logistic',
          eval_metric='auc',
          num_boost_round=5):

    print('Train features shape: {}'.format(X_train.shape))
    print('Train labels shape: {}'.format(y_train.shape))
    print('Validation features shape: {}'.format(X_val.shape))
    print('Validation labels shape: {}'.format(y_val.shape))        
    
    mlflow.set_tracking_uri(tracking_server_arn)
    mlflow.set_experiment(experiment_name)
    
    with mlflow.start_run(run_name=f"Training") as run:               
        mlflow.autolog()
             
        # Creating DMatrix(es)
        dtrain = xgboost.DMatrix(X_train, label=y_train)
        dval = xgboost.DMatrix(X_val, label=y_val)
        watchlist = [(dtrain, "train"), (dval, "validation")]
    
        print('')
        print (f'===Starting training with max_depth {max_depth}===')
        
        param_dist = {
            "max_depth": max_depth,
            "eta": eta,
            "gamma": gamma,
            "min_child_weight": min_child_weight,
            "verbosity": verbosity,
            "objective": objective,
            "eval_metric": eval_metric
        }        
    
        xgb = xgboost.train(
            params=param_dist,
            dtrain=dtrain,
            evals=watchlist,
            num_boost_round=num_boost_round)
    
        predictions = xgb.predict(dval)
    
        print ("Metrics for validation set")
        print('')
        print (pd.crosstab(index=y_val, columns=np.round(predictions),
                           rownames=['Actuals'], colnames=['Predictions'], margins=True))
        
        rounded_predict = np.round(predictions)
    
        val_accuracy = accuracy_score(y_val, rounded_predict)
        val_precision = precision_score(y_val, rounded_predict)
        val_recall = recall_score(y_val, rounded_predict)
    
        print("Accuracy Model A: %.2f%%" % (val_accuracy * 100.0))            
        print("Precision Model A: %.2f" % (val_precision))
        print("Recall Model A: %.2f" % (val_recall))
        
        # Log additional metrics, next to the default ones logged automatically
        mlflow.log_metric("Accuracy Model A", val_accuracy * 100.0)
        mlflow.log_metric("Precision Model A", val_precision)
        mlflow.log_metric("Recall Model A", val_recall)
        
        from sklearn.metrics import roc_auc_score
    
        val_auc = roc_auc_score(y_val, predictions)
        
        print("Validation AUC A: %.2f" % (val_auc))
        mlflow.log_metric("Validation AUC A", val_auc)
    
        model_file_path="/opt/ml/model/xgboost_model.bin"
        os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
        xgb.save_model(model_file_path)

    return xgb

In [ ]:
eta=0.3
max_depth=10

booster = train(X_train, y_train, X_val, y_val,
              eta=eta, 
              max_depth=max_depth)

Display the information about the trained model.

In [ ]:
booster

### Test the model with test data

In [ ]:
def test(featurizer_model, booster, X_test, y_test):

    mlflow.start_run(run_name=f"Testing")
    mlflow.autolog()
    X_test = featurizer_model.transform(X_test)
    y_test = y_test.values.reshape(-1)

    dtest = xgboost.DMatrix(X_test, label=y_test)
    test_predictions = booster.predict(dtest)
    
    print ("===Metrics for Test Set===")
    print('')
    print (pd.crosstab(index=y_test, columns=np.round(test_predictions), 
                                     rownames=['Actuals'], 
                                     colnames=['Predictions'], 
                                     margins=True)
          )
    print('')

    rounded_predict = np.round(test_predictions)

    accuracy = accuracy_score(y_test, rounded_predict)
    precision = precision_score(y_test, rounded_predict)
    recall = recall_score(y_test, rounded_predict)
    
    print('')
    print("Accuracy Model A: %.2f%%" % (accuracy * 100.0))
    print("Precision Model A: %.2f" % (precision))
    print("Recall Model A: %.2f" % (recall))
    
    mlflow.log_metric("Accuracy Model A", accuracy * 100.0)
    mlflow.log_metric("Precision Model A", precision)
    mlflow.log_metric("Recall Model A", recall)

    from sklearn.metrics import roc_auc_score

    auc = roc_auc_score(y_test, test_predictions)
    
    print("AUC A: %.2f" % (auc))
    mlflow.log_metric("AUC A",auc)
    
    mlflow.end_run()

In [ ]:
test(featurizer_model, booster, X_test, y_test)